<a href="https://colab.research.google.com/github/mhmtnzly/Autoscout24/blob/main/project_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import re
from google.colab import drive
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as sts
import itertools
%matplotlib inline
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/autoscout_data_2000.csv')
df = data.copy()

In [3]:
df['seller'].value_counts()
df['seller'] = df['seller'].str.extract('([A-Za-z]+\s*[a-z]+)')
df['seller'].unique()

array(['Dealer', 'Private seller', 'Diesel', 'Electric', 'Gasoline'],
      dtype=object)

In [4]:
df.drop((df[(df['seller']=='Diesel')|(df['seller']=='Electric')|(df['seller']=='Gasoline')].index).values, axis=0, inplace = True)

In [5]:
df.reset_index(drop=True,inplace=True)

In [6]:
df.columns

Index(['location', 'url', 'make', 'model', 'short_description', 'price',
       'mileage', 'gearbox', 'registration', 'fuel_type', 'power', 'seller',
       'body_type', 'type', 'drivetrain', 'seats', 'doors', 'country_version',
       'offer_number', 'warranty', 'Mileage', 'first_registration',
       'production_date', 'general_inspection', 'last_service',
       'full_service_history', 'non_smoker_vehicle', 'previous_owner',
       'last_timing_belt_change', 'Power', 'Gearbox', 'engine_size', 'gears',
       'cylinders', 'empty_weight', 'Fuel_type', 'fuel_consumption',
       'co2_emissions', 'emission_class', 'emissions_sticker',
       '\nComfort & Convenience\n', '\nEntertainment & Media\n',
       '\nSafety & Security\n', 'extras', '\ncolour', '\nmanufacturer_color',
       '\npaint', ' upholstery_colour ', ' upholstery '],
      dtype='object')

In [7]:
df.duplicated().sum()

0

In [8]:
my_df = df.copy()
my_df.rename(columns = {'\nComfort & Convenience\n':'Comfort_Convenience','location':'Location','fuel_type':'Fuel_Type','seller':'Seller','gears':'Gears','cylinders':'Cylinders',
                        'empty_weight':'Empty_weight', '\nSafety & Security\n': 'Safety_Security', '\ncolour': 'Colour', '\npaint': 'Paint'," upholstery_colour ": 'Upholstery_colour'," upholstery ": 'Upholstery', '\nEntertainment & Media\n':'Entertainment_Media'},inplace = True)
my_df.head()

,Location,url,make,model,short_description,price,mileage,gearbox,registration,Fuel_Type,...,emissions_sticker,Comfort_Convenience,Entertainment_Media,Safety_Security,extras,Colour,\nmanufacturer_color,Paint,Upholstery_colour,Upholstery
0,"LOON OP ZAND, NL",https://www.autoscout24.com/offers/audi-a3-2-0...,"\n,Audi ,\n","\n,A3,\n",2.0 FSI Ambition Pro Line,"[['\n', '€ 4,450.-', '\n']]","['', ['191,066 km']]","\n,Automatic,\n",08/2003,"\n,Gasoline,\n",...,"\n,None,\n","[['Air conditioning', 'Armrest', 'Automatic cl...","[['CD player', 'Radio']]","[['ABS', 'Alarm system', 'Central door lock', ...",[['Alloy wheels']],"\n4, Grey",None,"\npaint, Metallic","\nupholstery, Black","\n8, Cloth"
1,"HEERLEN, NL",https://www.autoscout24.com/offers/audi-a3-spo...,"\n,Audi ,\n","\n,A3,\n",Sportback 1.4 TFSI Ambiente/ECC/PDC//Lichtmeta...,"[['\n', '€ 12,950.-', '\n']]","['', ['94,758 km']]","\n,Automatic,\n",10/2013,"\n,Gasoline,\n",...,"\n,None,\n","[['Air conditioning', 'Automatic climate contr...","[['CD player', 'Radio']]","[['Alarm system', 'Central door lock', 'Centra...","[['Alloy wheels (16"")']]","\n4, White",None,None,"\nupholstery, Black","\n8, Cloth"
2,"BREDA, NL",https://www.autoscout24.com/offers/audi-a3-1-4...,"\n,Audi ,\n","\n,A3,\n",1.4 TFSI Attraction Pro Line [ Navi Airco Crui...,"[['\n', '€ 12,445.-', '\n']]","['', ['160,883 km']]","\n,Automatic,\n",05/2013,"\n,Gasoline,\n",...,"\n,None,\n","[['Air conditioning', 'Armrest', 'Cruise contr...","[['Bluetooth', 'CD player', 'Hands-free equipm...","[['ABS', 'Central door lock', 'Central door lo...","[['Alloy wheels', 'Spoiler']]","\n4, Black",None,"\npaint, Metallic",None,None
3,"BAARLE-NASSAU, NL",https://www.autoscout24.com/offers/audi-a1-1-4...,"\n,Audi ,\n","\n,A1,\n",1.4 TFSI Sportback Pro-Line Aut. S-Tronic Navi.,"[['\n', '€ 12,450.-', '\n']]","['', ['145,102 km']]","\n,Automatic,\n",06/2015,"\n,Gasoline,\n",...,"\n,None,\n","[['Air conditioning', 'Armrest', 'Automatic cl...","[['Bluetooth', 'CD player', 'On-board computer...","[['ABS', 'Central door lock with remote contro...","[['Alloy wheels (15"")']]","\n4, Grey",None,"\npaint, Metallic",None,None
4,"ZUTPHEN, NL",https://www.autoscout24.com/offers/audi-a1-s-l...,"\n,Audi ,\n","\n,A1,\n",S-Line | Automaat | Navigatie | Cruise-/Climat...,"[['\n', '€ 14,945.-', '\n']]","['', ['119,666 km']]","\n,Automatic,\n",10/2015,"\n,Gasoline,\n",...,"\n,None,\n","[['Air conditioning', 'Armrest', 'Automatic cl...","[['CD player', 'MP3', 'On-board computer', 'Ra...","[['ABS', 'Alarm system', 'Bi-Xenon headlights'...","[['Alloy wheels (17"")', 'Shift paddles', 'Spor...","\n4, Black",None,None,"\nupholstery, Black","\n8, Part leather"


* Comfort_Convenience: Features of car.
* model: Car's Model
* price: Car's price (€)

# None Data

In [9]:
def strNone(data):
  for column in data:
    my_df[column][my_df[column].str.contains('None', na = False)] = None
strNone(my_df)
my_df.head()

,Location,url,make,model,short_description,price,mileage,gearbox,registration,Fuel_Type,...,emissions_sticker,Comfort_Convenience,Entertainment_Media,Safety_Security,extras,Colour,\nmanufacturer_color,Paint,Upholstery_colour,Upholstery
0,"LOON OP ZAND, NL",https://www.autoscout24.com/offers/audi-a3-2-0...,"\n,Audi ,\n","\n,A3,\n",2.0 FSI Ambition Pro Line,"[['\n', '€ 4,450.-', '\n']]","['', ['191,066 km']]","\n,Automatic,\n",08/2003,"\n,Gasoline,\n",...,None,"[['Air conditioning', 'Armrest', 'Automatic cl...","[['CD player', 'Radio']]","[['ABS', 'Alarm system', 'Central door lock', ...",[['Alloy wheels']],"\n4, Grey",None,"\npaint, Metallic","\nupholstery, Black","\n8, Cloth"
1,"HEERLEN, NL",https://www.autoscout24.com/offers/audi-a3-spo...,"\n,Audi ,\n","\n,A3,\n",Sportback 1.4 TFSI Ambiente/ECC/PDC//Lichtmeta...,"[['\n', '€ 12,950.-', '\n']]","['', ['94,758 km']]","\n,Automatic,\n",10/2013,"\n,Gasoline,\n",...,None,"[['Air conditioning', 'Automatic climate contr...","[['CD player', 'Radio']]","[['Alarm system', 'Central door lock', 'Centra...","[['Alloy wheels (16"")']]","\n4, White",None,None,"\nupholstery, Black","\n8, Cloth"
2,"BREDA, NL",https://www.autoscout24.com/offers/audi-a3-1-4...,"\n,Audi ,\n","\n,A3,\n",1.4 TFSI Attraction Pro Line [ Navi Airco Crui...,"[['\n', '€ 12,445.-', '\n']]","['', ['160,883 km']]","\n,Automatic,\n",05/2013,"\n,Gasoline,\n",...,None,"[['Air conditioning', 'Armrest', 'Cruise contr...","[['Bluetooth', 'CD player', 'Hands-free equipm...","[['ABS', 'Central door lock', 'Central door lo...","[['Alloy wheels', 'Spoiler']]","\n4, Black",None,"\npaint, Metallic",None,None
3,"BAARLE-NASSAU, NL",https://www.autoscout24.com/offers/audi-a1-1-4...,"\n,Audi ,\n","\n,A1,\n",1.4 TFSI Sportback Pro-Line Aut. S-Tronic Navi.,"[['\n', '€ 12,450.-', '\n']]","['', ['145,102 km']]","\n,Automatic,\n",06/2015,"\n,Gasoline,\n",...,None,"[['Air conditioning', 'Armrest', 'Automatic cl...","[['Bluetooth', 'CD player', 'On-board computer...","[['ABS', 'Central door lock with remote contro...","[['Alloy wheels (15"")']]","\n4, Grey",None,"\npaint, Metallic",None,None
4,"ZUTPHEN, NL",https://www.autoscout24.com/offers/audi-a1-s-l...,"\n,Audi ,\n","\n,A1,\n",S-Line | Automaat | Navigatie | Cruise-/Climat...,"[['\n', '€ 14,945.-', '\n']]","['', ['119,666 km']]","\n,Automatic,\n",10/2015,"\n,Gasoline,\n",...,None,"[['Air conditioning', 'Armrest', 'Automatic cl...","[['CD player', 'MP3', 'On-board computer', 'Ra...","[['ABS', 'Alarm system', 'Bi-Xenon headlights'...","[['Alloy wheels (17"")', 'Shift paddles', 'Spor...","\n4, Black",None,None,"\nupholstery, Black","\n8, Part leather"


# Null Data Percentages and Dropping

In [10]:
def null_values(df):
    """a function to show null values with percentage"""
    nv=pd.concat([df.isnull().sum(), 100 * df.isnull().sum()/df.shape[0]],axis=1).rename(columns={0:'Missing_Records', 1:'Percentage (%)'})
    return nv[['Percentage (%)']].sort_values('Percentage (%)', ascending=False)
null_values(my_df)

,Percentage (%)
last_service,100.000000
\nmanufacturer_color,100.000000
emissions_sticker,99.783385
last_timing_belt_change,98.663741
previous_owner,93.073958
non_smoker_vehicle,92.011984
production_date,89.572960
country_version,87.313979
warranty,66.233437
Upholstery_colour,34.872141


* production_date, country_version was removed because almost 90 percent of the column is missing value.

In [11]:
my_df.drop(columns = ['url','Gearbox','previous_owner','production_date','country_version','\nmanufacturer_color',"Fuel_type","Paint","Upholstery_colour","Location",'last_service','emissions_sticker','last_timing_belt_change'], inplace = True)

# Variables

# Comfort_Convenience

In [12]:
my_df['Comfort_Convenience'] = my_df['Comfort_Convenience'].str.replace('contro\'\]','control')
my_df['Comfort_Convenience'] = my_df['Comfort_Convenience'].str.replace('whee\'\]','wheel')
my_df['Comfort_Convenience'] = my_df['Comfort_Convenience'].str.findall(r"\'(.+?)\'")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  


In [13]:
values = []
for liste in my_df['Comfort_Convenience']:
  for value in liste:
    values.append(value)
values = set(values)
def boolean_df(item_lists, unique_items):
# Create empty dict
    bool_dict = {}
    # Loop through all the tags
    for i, item in enumerate(unique_items):
        # Apply boolean mask
        bool_dict[item] = item_lists.apply(lambda x: 1 if item in x else 0)
    # Return the results as a dataframe
    return pd.DataFrame(bool_dict)
Comfort_Convenience = boolean_df(my_df['Comfort_Convenience'], values)
my_df = pd.concat([my_df,Comfort_Convenience],axis=1)
my_df.drop(columns = ['Comfort_Convenience'],inplace = True)
my_df.head()

,make,model,short_description,price,mileage,gearbox,registration,Fuel_Type,power,Seller,...,Lumbar support,Automatic climate control,Light sensor,Park Distance Control,Cruise control,Power windows,Armrest,Electric backseat adjustment,"Automatic climate control, 3 zones",Multi-function steering wheel
0,"\n,Audi ,\n","\n,A3,\n",2.0 FSI Ambition Pro Line,"[['\n', '€ 4,450.-', '\n']]","['', ['191,066 km']]","\n,Automatic,\n",08/2003,"\n,Gasoline,\n","110 kW (150 hp),",Dealer,...,0,1,0,0,0,1,1,0,0,0
1,"\n,Audi ,\n","\n,A3,\n",Sportback 1.4 TFSI Ambiente/ECC/PDC//Lichtmeta...,"[['\n', '€ 12,950.-', '\n']]","['', ['94,758 km']]","\n,Automatic,\n",10/2013,"\n,Gasoline,\n","92 kW (125 hp),",Dealer,...,0,1,1,0,0,1,0,0,0,0
2,"\n,Audi ,\n","\n,A3,\n",1.4 TFSI Attraction Pro Line [ Navi Airco Crui...,"[['\n', '€ 12,445.-', '\n']]","['', ['160,883 km']]","\n,Automatic,\n",05/2013,"\n,Gasoline,\n","90 kW (122 hp),",Dealer,...,0,0,0,0,1,1,1,0,0,1
3,"\n,Audi ,\n","\n,A1,\n",1.4 TFSI Sportback Pro-Line Aut. S-Tronic Navi.,"[['\n', '€ 12,450.-', '\n']]","['', ['145,102 km']]","\n,Automatic,\n",06/2015,"\n,Gasoline,\n","92 kW (125 hp),",Dealer,...,0,1,0,0,1,1,1,0,0,0
4,"\n,Audi ,\n","\n,A1,\n",S-Line | Automaat | Navigatie | Cruise-/Climat...,"[['\n', '€ 14,945.-', '\n']]","['', ['119,666 km']]","\n,Automatic,\n",10/2015,"\n,Gasoline,\n","92 kW (125 hp),",Dealer,...,1,1,0,0,1,1,1,0,0,1


In [14]:
def removing_column(df,threshold=0.005):
  print(df.columns[(df.sum()/df.shape[0])<threshold])
  return df.columns[(df.sum()/df.shape[0])<threshold]
my_df.drop(columns = removing_column(Comfort_Convenience), inplace = True)

Index(['Automatic climate control, 4 zones', 'Leather seats', 'Massage seats',
       'Park Distance Control'],
      dtype='object')


# Make - Model

In [15]:
my_df['make'].head()

0    \n,Audi ,\n
1    \n,Audi ,\n
2    \n,Audi ,\n
3    \n,Audi ,\n
4    \n,Audi ,\n
Name: make, dtype: object

In [16]:
my_df['make'].unique()

array(['\n,Audi ,\n', '\n,BMW ,\n', '\n,Ford ,\n', '\n,Mercedes-Benz ,\n',
       '\n,Opel ,\n', '\n,Volkswagen ,\n', '\n,Renault ,\n',
       '\n,Citroen ,\n', '\n,Chevrolet ,\n', '\n,Dacia ,\n',
       '\n,Fiat ,\n', '\n,Honda ,\n', '\n,Hyundai ,\n', '\n,Kia ,\n',
       '\n,Mazda ,\n', '\n,Peugeot ,\n', '\n,Skoda ,\n', '\n,Toyota ,\n',
       '\n,Tesla ,\n', '\n,Volvo ,\n'], dtype=object)

In [17]:
my_df['make'] = my_df['make'].str.extract(r"\,(.+?)\s")

In [18]:
my_df['model'] = my_df['model'].str.extract(r"\,(.+?)\,")

In [19]:
my_df['make-model'] = my_df['make']+'-'+my_df['model']

In [20]:
my_df.drop(columns = ['make','model'],inplace = True)
my_df.shape

(71094, 74)

In [ ]:
names = my_df['make-model'].value_counts()[my_df['make-model'].value_counts()<30].index.values
for name in names:
  my_df.drop(my_df[my_df['make-model']==name].index.values, axis = 0, inplace = True)
my_df.reset_index(drop = True, inplace = True)

In [ ]:
my_df.shape

# Price

In [ ]:
my_df['price'] = my_df['price'].str.extract(r"\€(.+?)\.")
my_df['price'] = my_df['price'].str.replace(',','')

In [ ]:
my_df['price'] = my_df['price'].apply(pd.to_numeric, errors='ignore')

In [ ]:
my_df['price'].head()

# Emission_class

In [ ]:
my_df['emission_class'].unique()

In [ ]:
my_df['emission_class'] = my_df['emission_class'].str.extract(r"\n,(.+?),\n")

In [ ]:
emission_class = my_df['emission_class'].str.get_dummies().add_prefix('emission_class_')

In [ ]:
my_df.drop('emission_class',inplace = True, axis = 1)

# Doors

In [ ]:
my_df['doors'].unique()

In [ ]:
my_df['doors'] = my_df['doors'].str.extract(r"(\d+)")

In [ ]:
my_df['doors'] = my_df['doors'].apply(pd.to_numeric, errors='ignore')

# offer_number

offer_number is removed because it is considered not useful.

In [ ]:
my_df.drop(columns = ['offer_number'],inplace = True)

# warranty

In [ ]:
my_df['warranty'].unique()

In [ ]:
my_df['warranty'] = my_df['warranty'].str.extract(r"\n,(.+?),\n")
my_df['warranty'].unique()

In [ ]:
my_df['warranty'].fillna('0', inplace = True)

In [ ]:
my_df[my_df['warranty']=='Yes']['warranty']

In [ ]:
my_df.loc[my_df['warranty']=='Yes',['warranty']] = '6'

In [ ]:
my_df['warranty'] = my_df['warranty'].str.extract(r"(\d+)").apply(pd.to_numeric, errors='ignore')

In [ ]:
my_df.rename(columns = {'warranty':'warranty_months'},inplace = True)

# Mileage

In [ ]:
my_df['Mileage'] = my_df['Mileage'].str.replace(r'[\[\]\'\\n,\s]','')

In [ ]:
my_df['Mileage'].unique()

In [ ]:
my_df.drop(columns = ['Mileage'],inplace = True)

# first_registration

In [ ]:
my_df['first_registration_years'] = (pd.Timestamp.now() - pd.to_datetime(df['first_registration'],format='%m/%Y',errors='coerce')) / np.timedelta64(1,'Y')

In [ ]:
my_df.loc[my_df['first_registration_years']<0,['first_registration_years','Seller','price','mileage']]

In [ ]:
indexes = (my_df.loc[my_df['first_registration_years']<0].index).values
my_df = my_df.drop(indexes,axis = 0)

In [ ]:
my_df.reset_index(drop=True,inplace=True)
my_df.drop(columns = ['first_registration'], inplace = True)

In [ ]:
my_df.shape

# general_inspection

In [ ]:
my_df['general_inspection'].unique()

In [ ]:
my_df['general_inspection'].value_counts()

In [ ]:
my_df['general_inspection'] = my_df['general_inspection'].str.replace('New','02/2022')

In [ ]:
my_df['general_inspection'].unique()

In [ ]:
my_df['general_inspection_new'] = (pd.to_datetime(df['general_inspection'],format='%m/%Y',errors='coerce') - pd.Timestamp.now()) < np.timedelta64(6,'M')

In [ ]:
my_df['general_inspection_new'] = my_df['general_inspection_new'].apply(lambda x: 1 if x==True else 0)

In [ ]:
my_df.drop(columns = ['general_inspection'],inplace = True)

# Fuel_consumption

In [ ]:
my_df['fuel_consumption']=my_df['fuel_consumption'].str.extract('(\d*\.?\d)')#\w+.(\d*) (\d*.?\d)
my_df['fuel_consumption']

In [ ]:
my_df['fuel_consumption'] = my_df['fuel_consumption'].apply(pd.to_numeric, errors='ignore')
my_df['fuel_consumption'].head()

In [ ]:
indexes = my_df[(my_df['fuel_consumption']<=3) | (my_df['fuel_consumption']>=18)]['fuel_consumption'].index.values
my_df['fuel_consumption'][indexes] = None

# extras

In [ ]:
my_df['extras'].unique()

In [ ]:
my_df['extras'] = my_df['extras'].str.replace('Contro\'','Control\'')
my_df['extras'] = my_df['extras'].str.replace(',','')
my_df['extras'] =  my_df['extras'].str.findall(r'\'(.+?)\'')
my_df['extras'].head()

In [ ]:
values = []
for liste in my_df['extras']:
  for value in liste:
    values.append(value)
values = set(values)
def boolean_df(item_lists, unique_items):
# Create empty dict
    bool_dict = {}
    # Loop through all the tags
    for i, item in enumerate(unique_items):
        # Apply boolean mask
        bool_dict[item] = item_lists.apply(lambda x: 1 if item in x else 0)
    # Return the results as a dataframe
    return pd.DataFrame(bool_dict)
extras_features = boolean_df(my_df['extras'], values)
my_df = pd.concat([my_df,extras_features],axis=1)
my_df.drop(columns = ['extras'],inplace = True)
my_df.tail()

In [ ]:
def removing_column(df,threshold=0.005):
  return df.columns[(df.sum()/df.shape[0])<threshold]
removing_column(extras_features)
my_df.drop(columns = removing_column(extras_features), inplace = True)

# co2_emissions

In [ ]:
my_df['co2_emissions'] = my_df['co2_emissions'].str.extract('(\d+)')
my_df['co2_emissions'].head()

In [ ]:
my_df['co2_emissions'] = my_df['co2_emissions'].apply(pd.to_numeric, errors='ignore')

# power

In [ ]:
my_df['power'] = my_df['power'].str.extract('(^\d*.?\d)')
my_df['power'] = my_df['power'].apply(pd.to_numeric, errors='ignore')
my_df['power'].head()

# body_type

In [ ]:
my_df['body_type'] = my_df['body_type'].str.replace('\n','').str.replace(',','')
my_df['body_type'].head()

In [ ]:
my_df['body_type'].unique()

# type

In [ ]:
my_df['type'] = my_df['type'].str.replace('\n','').str.replace(',','')

In [ ]:
my_df['type'] = my_df['type'].replace('Antique / Classic','Classic')
my_df['type'].unique()

# drivetrain

In [ ]:
my_df['drivetrain'] = my_df['drivetrain'].str.replace('\n','').str.replace(',','')

In [ ]:
my_df['drivetrain'].unique()

# seats

In [ ]:
my_df['seats'] = my_df['seats'].str.extract("(\d+)").apply(pd.to_numeric, errors='ignore')

# registration

* Since there are two identical registration I dropped this one.

In [ ]:
df[['registration','first_registration']].head()

In [ ]:
my_df.drop(['registration'], axis =1, inplace =True)

In [ ]:
my_df[my_df['first_registration_years']<0]

# Gears

In [ ]:
my_df['Gears'] = my_df.Gears.str.extract("(\d)")

In [ ]:
my_df['Gears'] = my_df['Gears'].apply(pd.to_numeric, errors='ignore')

In [ ]:
my_df['Gears'] = my_df.Gears.replace([6.5, 5.5, 7.5, 4.5], [6, 5, 7, 4])

# Cylinders

In [ ]:
my_df['Cylinders'] = my_df.Cylinders.str.extract("(\d+)").apply(pd.to_numeric, errors='ignore')

In [ ]:
my_df['Cylinders'] = my_df.Cylinders.replace([12, 7], 4)

# Empty_weight

In [ ]:
my_df['Empty_weight'] = my_df.Empty_weight.str.extract(r"\n,(.+?)\s")
my_df['Empty_weight'] = my_df.Empty_weight.str.replace(',','').apply(pd.to_numeric, errors='ignore')

In [ ]:
my_df['Empty_weight'] = my_df.Empty_weight.replace([1,112,180], None)

# Fuel_type

In [ ]:
my_df['Fuel_Type'] = my_df['Fuel_Type'].str.extract('(\w+.\w+|\w+)')

In [ ]:
my_df['Fuel_Type'] = my_df.Fuel_Type.replace(["Electric/Gasoline", "Electric/Diesel", "Hydrogen", "Ethanol", "Others"], ["Hybrid", "Hybrid", "Gasoline", "Gasoline", "Gasoline"])

# Colour

In [ ]:
my_df['Colour'] = my_df.Colour.str.extract("(\w{2,10})")
my_df.Colour.unique()

In [ ]:
my_df['Colour'] = my_df.Colour.replace(["Gold", "Bronze"], "Grey")

# Upholstery

In [ ]:
my_df['Upholstery'] = my_df.Upholstery.str.extract('(\w+\s\w+|\w\w+)')

In [ ]:
my_df['Upholstery'] = my_df.Upholstery.replace(["Other", "Velour", "alcantara", "Part leather", "Full leather"], [None, "Cloth", "Part/Full Leather", "Part/Full Leather", "Part/Full Leather"])

# Safety_Security

In [ ]:
my_df['Safety_Security'] = my_df['Safety_Security'].str.replace('Contro\'\]','Control')
my_df['Safety_Security'] = my_df['Safety_Security'].str.replace('contro\'\]','control')
my_df['Safety_Security'] = my_df['Safety_Security'].str.findall(r'\'(.+?)\'')

In [ ]:
values = []
for liste in my_df['Safety_Security']:
  for value in liste:
    values.append(value)
values = set(values)
def boolean_df(item_lists, unique_items):
# Create empty dict
    bool_dict = {}
    # Loop through all the tags
    for i, item in enumerate(unique_items):
        # Apply boolean mask
        bool_dict[item] = item_lists.apply(lambda x: 1 if item in x else 0)
    # Return the results as a dataframe
    return pd.DataFrame(bool_dict)
Safety_Security = boolean_df(my_df['Safety_Security'], values)
my_df = pd.concat([my_df,Safety_Security],axis=1)
my_df.drop(columns = ['Safety_Security'],inplace = True)
my_df.tail()

In [ ]:
def removing_column(df,threshold=0.005):
  print(df.columns[(df.sum()/df.shape[0])<threshold])
  return df.columns[(df.sum()/df.shape[0])<threshold]
my_df.drop(columns = removing_column(Safety_Security), inplace = True)

# Seller

In [ ]:
my_df['Seller'] = my_df.Seller.str.extract('(\w+\s\w+|\w\w\w\w\w\w)')
my_df.head()

# short_describtion

* The pattern that short_description has unpredictable so I dropped it.

In [ ]:
my_df.drop(columns = ['short_description'], inplace = True)

# Entertainment_Media

In [ ]:
my_df[["Entertainment_Media"]].value_counts(dropna = False, normalize = True)

In [ ]:
#Cleaning unneccessary symbols
my_df['Entertainment_Media'] = my_df['Entertainment_Media'].str.findall(r"\'(.+?)\'")

In [ ]:
values = []
for liste in my_df['Entertainment_Media']:
  for value in liste:
    values.append(value)
values = set(values)
def boolean_df(item_lists, unique_items):
# Create empty dict
    bool_dict = {}
    # Loop through all the tags
    for i, item in enumerate(unique_items):
        # Apply boolean mask
        bool_dict[item] = item_lists.apply(lambda x: 1 if item in x else 0)
    # Return the results as a dataframe
    return pd.DataFrame(bool_dict)
Entertainment_Media = boolean_df(my_df['Entertainment_Media'], values)
my_df = pd.concat([my_df,Entertainment_Media],axis=1)
my_df.drop(columns = ['Entertainment_Media'],inplace = True)
my_df.tail()

In [ ]:
def removing_column(df,threshold=0.005):
  print(df.columns[(df.sum()/df.shape[0])<threshold])
  return df.columns[(df.sum()/df.shape[0])<threshold]
my_df.drop(columns = removing_column(Entertainment_Media), inplace = True)

# mileage

In [ ]:
my_df['mileage'] = df['mileage'].str.extract('(\d+,\d+|\d+)')
my_df['mileage'] = my_df['mileage'].str.replace(',','')
my_df['mileage'] = my_df['mileage'].apply(pd.to_numeric, errors='ignore')

# gearbox

In [ ]:
my_df[["gearbox"]].value_counts(dropna = False, normalize = True)

In [ ]:
my_df['gearbox'] = my_df['gearbox'].str.extract(r"\,(.+?)\,")

# full_service_history

In [ ]:
my_df[["full_service_history"]].value_counts(dropna = False, normalize = True)

In [ ]:
my_df['full_service_history'] = my_df['full_service_history'].str.extract(r"\n,(.+?),\n")
my_df['full_service_history'].fillna('No', inplace = True)

# non_smoker_vehicle

In [ ]:
my_df[["non_smoker_vehicle"]].value_counts(dropna = False, normalize = True)

In [ ]:
my_df['non_smoker_vehicle'] = my_df.non_smoker_vehicle.str.extract('(\w{2,3})')
my_df['non_smoker_vehicle'].fillna('No', inplace = True)
my_df['non_smoker_vehicle'].head()

# Power

* There are 2 namesake 'power' column so I dropped it.

In [ ]:
my_df[["Power"]].value_counts(dropna = False, normalize = True)

In [ ]:
my_df['Power'] = my_df['Power'].str.extract('(\w{1,4})')
my_df['Power'] = my_df['Power'].apply(pd.to_numeric, errors='ignore')
my_df['Power'].head()

In [ ]:
my_df.drop(columns = ['power'], inplace = True)

# engine_size

In [ ]:
my_df[["engine_size"]].value_counts(dropna = False, normalize = True)

In [ ]:
my_df['engine_size'] = my_df.engine_size.str.extract('(\d+\,\d+)')
my_df['engine_size'] = my_df.engine_size.str.replace(',','').apply(pd.to_numeric, errors='ignore')

In [ ]:
my_df.drop((my_df[my_df['price'].isnull()].index).values, axis=0, inplace = True)
my_df.drop((my_df[(my_df['mileage']<1000) & (my_df['first_registration_years']>2)].index).values, axis = 0, inplace = True)
my_df.reset_index(inplace = True, drop = True)

In [ ]:
miss_df = my_df.copy()
miss_df.head()

# Missing Value

In [ ]:
indexes = miss_df[(miss_df['Fuel_Type']!="Electric")&(miss_df['co2_emissions']==0)]['co2_emissions'].index.values
miss_df['co2_emissions'][indexes] = None

In [ ]:
fry_qcut = miss_df[['first_registration_years']]
miss_df['fry_qcut'] = pd.qcut(fry_qcut['first_registration_years'], q = 10)

In [ ]:
ew_qcut = miss_df[['Empty_weight']]
miss_df['ew_qcut'] = pd.qcut(ew_qcut['Empty_weight'], q = 10)

In [ ]:
es_qcut = miss_df[['engine_size']]
miss_df['es_qcut'] = pd.qcut(es_qcut['engine_size'], q = 10)

In [ ]:
p_qcut = miss_df[['Power']]
miss_df['p_qcut'] = pd.qcut(p_qcut['Power'], q = 10)

In [ ]:
co_qcut = miss_df[['co2_emissions']]
miss_df['co_qcut'] = pd.qcut(co_qcut['co2_emissions'], q = 5)

In [ ]:
fc_qcut = miss_df[['fuel_consumption']]
miss_df['fc_qcut'] = pd.qcut(fc_qcut['fuel_consumption'], q = 10)

In [ ]:
m_qcut = miss_df[['mileage']]
miss_df['m_qcut'] = pd.qcut(m_qcut['mileage'], q = 20)

In [ ]:
import missingno as msno
msno.bar(miss_df[['price', 'mileage', 'gearbox', 'Fuel_Type', 'Seller', 'body_type',
       'type', 'drivetrain', 'seats', 'doors', 'warranty_months',
       'full_service_history', 'non_smoker_vehicle', 'Power',
       'engine_size', 'Gears', 'Cylinders', 'Empty_weight',
       'fuel_consumption', 'co2_emissions', 'Colour', 'Upholstery','make-model', 'first_registration_years', 'general_inspection_new']]);

In [ ]:
miss_df[['price', 'mileage', 'gearbox', 'Fuel_Type', 'Seller', 'body_type',
       'type', 'drivetrain', 'seats', 'doors', 'warranty_months',
       'full_service_history', 'non_smoker_vehicle', 'Power',
       'engine_size', 'Gears', 'Cylinders', 'Empty_weight',
       'fuel_consumption', 'co2_emissions', 'Colour', 'Upholstery','make-model', 'first_registration_years', 'general_inspection_new']].isnull().sum()

In [ ]:
def fill_most(df, group_col, col_name):
    for group in list(df[group_col].unique()):
        cond = df[group_col]==group
        grp_inx = list(df[cond][col_name].index)
        mode = list(df[cond][col_name].mode())
        if mode != []:
            df[col_name].iloc[grp_inx] = df[col_name].iloc[grp_inx].fillna(df[cond][col_name].mode()[0])
        else:
            df[col_name].iloc[grp_inx] = df[col_name].iloc[grp_inx].fillna(df[col_name].mode()[0])
def fill(df, group_col1, group_col2, col_name, method): # method can be "mode" or "median" or "ffill/bfill"
    if method == "mode":
        for group1 in list(df[group_col1].unique()):
            for group2 in list(df[group_col2].unique()):
                cond1 = df[group_col1]==group1
                cond2 = (df[group_col1]==group1) & (df[group_col2]==group2)
                grp_inx = list(df[cond2][col_name].index)
                mode1 = list(df[cond1][col_name].mode())
                mode2 = list(df[cond2][col_name].mode())
                if (mode1 != []) and (mode2 != []):
                    df[col_name].iloc[grp_inx] = df[col_name].iloc[grp_inx].fillna(df[cond2][col_name].mode()[0]).fillna(df[cond1][col_name].mode()[0])
                elif mode1 != []:
                    df[col_name].iloc[grp_inx] = df[col_name].iloc[grp_inx].fillna(df[cond1][col_name].mode()[0])
                else:
                    df[col_name].iloc[grp_inx] = df[col_name].iloc[grp_inx].fillna(df[col_name].mode()[0])  


In [ ]:
fill_most(miss_df,'make-model','body_type')
fill_most(miss_df,'make-model','drivetrain')
fill_most(miss_df,'make-model','type')
fill_most(miss_df,'make-model','Colour')
fill_most(miss_df,'make-model','Upholstery')
fill(miss_df,'make-model','body_type', 'gearbox', 'mode')

boolean = True
count = 0
while boolean:
  count += 1
  total_miss_numbers = miss_df[['gearbox','seats','Power','doors','warranty_months','engine_size','Gears','Cylinders','Empty_weight','fuel_consumption','co2_emissions','general_inspection_new']].isnull().sum().sum()
  if total_miss_numbers == 0:
    boolean = False
  if count == 10:
    boolean = False
  else:
    miss_df['doors'] = miss_df['doors'].fillna(miss_df.groupby(['make-model'])['doors'].transform('median'))
    miss_df['seats'] = miss_df['seats'].fillna(miss_df.groupby(['make-model'])['seats'].transform('median'))
    miss_df['Power'] = miss_df['Power'].fillna(miss_df.groupby(['es_qcut','ew_qcut'])['Power'].transform('median'))
    miss_df['Power'] = miss_df['Power'].fillna(miss_df.groupby(['fc_qcut','ew_qcut'])['Power'].transform('median'))
    miss_df['Power'] = miss_df['Power'].fillna(miss_df.groupby(['fc_qcut','es_qcut'])['Power'].transform('median'))
    miss_df['Power'] = miss_df['Power'].fillna(miss_df.groupby(['ew_qcut'])['Power'].transform('median'))
    miss_df['Power'] = miss_df['Power'].fillna(miss_df.groupby(['es_qcut'])['Power'].transform('median'))
    miss_df['Power'] = miss_df['Power'].fillna(miss_df.groupby(['make-model'])['Power'].transform('median'))
    miss_df['engine_size'] = miss_df['engine_size'].fillna(miss_df.groupby(['Cylinders'])['engine_size'].transform('median'))
    miss_df['Gears'] = miss_df['Gears'].fillna(miss_df.groupby(['make-model'])['Gears'].transform('median'))
    miss_df['Cylinders'] = miss_df['Cylinders'].fillna(miss_df.groupby(['make-model'])['Cylinders'].transform('median'))
    miss_df['Cylinders'] = miss_df['Cylinders'].fillna(miss_df.groupby(['es_qcut'])['Cylinders'].transform('median'))
    miss_df['Cylinders'] = miss_df['Cylinders'].fillna(miss_df.groupby(['Gears'])['Cylinders'].transform('median'))
    miss_df['Empty_weight'] = miss_df['Empty_weight'].fillna(miss_df.groupby(['make-model'])['Empty_weight'].transform('median'))
    miss_df['fuel_consumption'] = miss_df['fuel_consumption'].fillna(miss_df.groupby(['es_qcut'])['fuel_consumption'].transform('median'))
    miss_df['fuel_consumption'] = miss_df['fuel_consumption'].fillna(miss_df.groupby(['co_qcut'])['fuel_consumption'].transform('median'))
    miss_df['fuel_consumption'] = miss_df['fuel_consumption'].fillna(miss_df.groupby(['Cylinders'])['fuel_consumption'].transform('median'))
    miss_df['fuel_consumption'] = miss_df['fuel_consumption'].fillna(miss_df.groupby(['ew_qcut'])['fuel_consumption'].transform('median'))
    miss_df['co2_emissions'] = miss_df['co2_emissions'].fillna(miss_df.groupby(['fc_qcut'])['co2_emissions'].transform('median'))
    miss_df['co2_emissions'] = miss_df['co2_emissions'].fillna(miss_df.groupby(['Cylinders'])['co2_emissions'].transform('median'))
    miss_df['co2_emissions'] = miss_df['co2_emissions'].fillna(miss_df.groupby(['make-model'])['co2_emissions'].transform('median'))
    miss_df['co2_emissions'] = miss_df['co2_emissions'].fillna(miss_df.groupby(['p_qcut'])['co2_emissions'].transform('median'))
    miss_df['first_registration_years'] = miss_df['first_registration_years'].fillna(miss_df.groupby(['mileage'])['first_registration_years'].transform('median'))
    miss_df['first_registration_years'] = miss_df['first_registration_years'].fillna(miss_df.groupby(['co_qcut','m_qcut'])['first_registration_years'].transform('median'))
    miss_df['first_registration_years'] = miss_df['first_registration_years'].fillna(miss_df.groupby(['Fuel_Type'])['first_registration_years'].transform('median'))    
    miss_df['mileage'] = miss_df['mileage'].fillna(miss_df.groupby(['fry_qcut','general_inspection_new'])['mileage'].transform('mean'))
    miss_df['mileage'] = miss_df['mileage'].fillna(miss_df.groupby(['fry_qcut'])['mileage'].transform('mean'))
    miss_df['mileage'] = miss_df['mileage'].fillna(miss_df.groupby(['general_inspection_new'])['mileage'].transform('mean'))


In [ ]:
miss_df[['price', 'mileage', 'gearbox', 'Fuel_Type', 'Seller', 'body_type',
       'type', 'drivetrain', 'seats', 'doors', 'warranty_months',
       'full_service_history', 'non_smoker_vehicle', 'Power',
       'engine_size', 'Gears', 'Cylinders', 'Empty_weight',
       'fuel_consumption', 'co2_emissions', 'Colour', 'Upholstery','make-model', 'first_registration_years', 'general_inspection_new']].corr()

In [ ]:
miss_df[['price', 'mileage', 'gearbox', 'Fuel_Type', 'Seller', 'body_type',
       'type', 'drivetrain', 'seats', 'doors', 'warranty_months',
       'full_service_history', 'non_smoker_vehicle', 'Power',
       'engine_size', 'Gears', 'Cylinders', 'Empty_weight',
       'fuel_consumption', 'co2_emissions', 'Colour', 'Upholstery','make-model', 'first_registration_years', 'general_inspection_new']].isnull().sum()

In [ ]:
Fuel_Type = pd.get_dummies(miss_df["Fuel_Type"], drop_first=True).add_prefix("Fuel_Type_")
Colour = pd.get_dummies(miss_df["Colour"], drop_first=True).add_prefix("Colour_")
Upholstery = pd.get_dummies(miss_df["Upholstery"], drop_first=True).add_prefix("Upholstery_")
Seller = pd.get_dummies(miss_df["Seller"], drop_first=True).add_prefix("Seller_")
gearbox = pd.get_dummies(miss_df["gearbox"], drop_first=True).add_prefix("gearbox_")
body_type = pd.get_dummies(miss_df["body_type"], drop_first=True).add_prefix("body_type_")
Type = pd.get_dummies(miss_df["type"], drop_first=True).add_prefix("type_")
drivetrain = pd.get_dummies(miss_df["drivetrain"], drop_first=True).add_prefix("drivetrain_")
full_service_history = pd.get_dummies(miss_df["full_service_history"], drop_first=True).add_prefix("full_service_history_")
non_smoker_vehicle = pd.get_dummies(miss_df["non_smoker_vehicle"], drop_first=True).add_prefix("non_smoker_vehicle_")
make_model = pd.get_dummies(miss_df["make-model"], drop_first=True).add_prefix("make-model_")

In [ ]:
total = pd.concat([Fuel_Type,Colour,Upholstery,Seller,gearbox,body_type,Type,drivetrain,full_service_history,non_smoker_vehicle,make_model],axis=1)
total.shape

In [ ]:
miss_df = pd.concat([miss_df,total],axis=1)

In [ ]:
miss_df.drop(columns = ["Fuel_Type","Colour","Upholstery","Seller","gearbox","body_type","type","drivetrain","full_service_history","non_smoker_vehicle","make-model"], inplace = True)

In [ ]:
miss_df.drop(columns = ['price', 'mileage','seats', 'doors', 'warranty_months', 'Power',
       'engine_size', 'Gears', 'Cylinders', 'Empty_weight',
       'fuel_consumption', 'co2_emissions', 'first_registration_years', 'general_inspection_new'])

In [ ]:
corr_matrix = miss_df.corr().abs()
high_corr_var=np.where(corr_matrix>0.9)
high_corr_var=[(corr_matrix.columns[x],corr_matrix.columns[y]) for x,y in zip(*high_corr_var) if x!=y and x<y]

In [ ]:
for i in range(len(high_corr_var)):
  miss_df.drop(columns = [high_corr_var[i][0]], inplace = True)

# Data Featuring

In [ ]:
miss_df["mileage/years"] = miss_df["mileage"]/miss_df["first_registration_years"]

In [ ]:
miss_df["total_consumption"] = miss_df["fuel_consumption"]*miss_df["mileage"]/100

In [ ]:
miss_df["total_co2_emissions"] = miss_df["mileage"] * miss_df["co2_emissions"]

# Functions_Outliers

In [ ]:
def capping_outliers(s, whisker=1.5):
  # replace outliers with upper_bound and lower_bound values
  indexes = s.dropna().index
  col = s.dropna()
  iqr = (np.quantile(col, 0.95))-(np.quantile(col, 0.05))
  upper_bound = np.quantile(col, 0.95)+(whisker*iqr)
  lower_bound = np.quantile(col, 0.05)-(whisker*iqr)
  upper_bound = upper_bound if upper_bound < col.max() else col.max(),
  lower_bound = lower_bound if lower_bound > col.min() else col.min(),

  col = np.where(col > upper_bound, upper_bound,col)
  col = np.where(col < lower_bound, lower_bound,col)
  plt.figure(figsize=(8,3))
  sns.boxplot(x= col)
  plt.show()
  s[indexes]=col
  return s

In [ ]:
miss_df['Gears'] = capping_outliers(miss_df['Gears'])
miss_df['Cylinders'] = capping_outliers(miss_df['Cylinders'])
miss_df['Empty_weight'] = capping_outliers(miss_df['Empty_weight'])
miss_df['mileage'] = capping_outliers(miss_df['mileage'])
miss_df['seats'] = capping_outliers(miss_df['seats'])
miss_df['doors'] = capping_outliers(miss_df['doors'])
miss_df['warranty_months'] = capping_outliers(miss_df['warranty_months'])
miss_df['Power'] = capping_outliers(miss_df['Power'])
miss_df['engine_size'] = capping_outliers(miss_df['engine_size'])
miss_df['fuel_consumption'] = capping_outliers(miss_df['fuel_consumption'])
miss_df['co2_emissions'] = capping_outliers(miss_df['co2_emissions'])
miss_df['first_registration_years'] = capping_outliers(miss_df['first_registration_years'])
miss_df["total_co2_emissions"] = capping_outliers(miss_df["total_co2_emissions"])
miss_df["total_consumption"] = capping_outliers(miss_df["total_consumption"])
miss_df["mileage/years"] = capping_outliers(miss_df["mileage/years"])

In [ ]:
from sklearn.neighbors import LocalOutlierFactor

In [ ]:
clf = LocalOutlierFactor(n_neighbors = 8, contamination = 0.1)
clf.fit_predict(miss_df[['price', 'mileage','seats', 'doors', 'warranty_months', 'Power',
       'engine_size', 'Gears', 'Cylinders', 'Empty_weight',
       'fuel_consumption', 'co2_emissions', 'first_registration_years', 'general_inspection_new',"total_co2_emissions","total_consumption","mileage/years"]])
X_noncategorical_scores=clf.negative_outlier_factor_

In [ ]:
threshold_value = np.sort(X_noncategorical_scores)[18]
miss_df.drop(miss_df[X_noncategorical_scores<threshold_value].index, axis = 0, inplace = True)

# Visulation

In [ ]:
miss_df.Gears.plot.box(vert=False);

In [ ]:
miss_df.Cylinders.plot.box(vert=False);

In [ ]:
miss_df.Empty_weight.plot.box(vert=False);

In [ ]:
miss_df.mileage.plot.box(vert=False);

In [ ]:
miss_df.seats.plot.box(vert=False);

In [ ]:
miss_df.doors.plot.box(vert=False);

In [ ]:
miss_df.warranty_months.plot.box(vert=False);

In [ ]:
miss_df.Power.plot.box(vert=False);

In [ ]:
miss_df.engine_size.plot.box(vert=False);

In [ ]:
miss_df.fuel_consumption.plot.box(vert=False);

In [ ]:
miss_df.co2_emissions.plot.box(vert=False);

In [ ]:
miss_df.first_registration_years.plot.box(vert=False);

In [ ]:
miss_df.to_csv('car_data.csv')